In [2]:
import pandas as pd
import cleaningtools as ct 
import file_config as fconfig
import curate_config as config
import numpy as np
import math
from tqdm.notebook import tqdm_notebook as tqdm
tqdm.pandas()
from tqdm import tqdm
import os
import file_config as fconfig
import pandas as pd
from IPython.display import display

check for pandas
pandas is properly installed
check for tqdm
tqdm is properly installed
check for numpy
numpy is properly installed
check for openpyxl
openpyxl is properly installed
check for xlwings
xlwings is properly installed


In [3]:
dir='/Users/sebastiancoombs/Library/CloudStorage/GoogleDrive-sebastianbcoombs@gmail.com/Shared drives/AMC Projects/_AZ_Kay/_Master Databases/'
files= os.listdir(dir)

In [45]:
files=[f for f in files if 'drill' in f]
files=[f for f in files if f.endswith('.csv')]
files=[f for f in files if 'merged' not in f]
files=[f for f in files if 'shift' not in f]

samples=[f for f in files if 'samples' in f]

keys=sorted([k.split(' ')[1] for k in samples]+['geochemical'])
all_samples=[k for k in samples if k.split(' ')[1] =='samples']
d_list=[]
keys.remove('samples')
print(keys,all_samples)
for k in keys:
    merge_files=[f for f in files if k in f ]
    globed=''.join(merge_files)
    if 'geochemical' in globed or 'hyp-pkg' in globed:
        merge_files+=all_samples
    print('---------------------------------------------------')
    merg_files=sorted(merge_files)
    print(merge_files)
    base=merg_files[0].split(' ')
    basename=' '.join(base[:2])
    print(basename)

    merged=pd.DataFrame()
    if len(merge_files)<=2:
        s_file=merge_files[-1]
        s_name=merge_files[-1].split('.')[0]
        s_data=pd.read_csv(dir+s_file,low_memory=False)
        s_col=s_data.filter(like='sample').columns[0]


        d_file=merge_files[0]
        d_name=merge_files[0].split('.')[0]
        d_data=pd.read_csv(dir+d_file,low_memory=False)
        d_col=d_data.filter(like='sample').columns[0]
        d_sub=d_name.split(' ')[2]
        
        if 'terraspec' in d_file:
            s_col=s_data.filter(like='file_name').columns[0]

        if 'geochemical' in merged:
            s_sub=s_name.split(' ')[1]
        else:
            s_sub=s_name.split(' ')[2]


        
        print(f'Merging {s_name}  with {d_name} data')
        print('using columns')
        print(s_col,';',d_col)
        #data=pd.concat([s_data.set_index(s_col),d_data.set_index(d_col)], axis=1, join='inner')
        data=pd.merge(  
                s_data.drop_duplicates(subset=[s_col],keep='first'),
                d_data.drop_duplicates(subset=[d_col],keep='first'),
        left_on=s_col,
        right_on=d_col,
        how='inner',
        suffixes=['_'+s_sub,'_'+d_sub],
        )
    
    
    if len(merge_files)>2:

        s_file=merge_files[-1]
        s_name=merge_files[-1].split('.')[0]
        s_data=pd.read_csv(dir+s_file,low_memory=False)
        s_col=s_data.filter(like='sample').columns[0]

        d_file=merge_files[0]
        d_name=merge_files[0].split('.')[0]
        d_data=pd.read_csv(dir+d_file,low_memory=False)
        d_col=d_data.filter(like='sample').columns[0]

        d_file2=merge_files[1]
        d_name2=merge_files[1].split('.')[0]
        d_data2=pd.read_csv(dir+d_file2,low_memory=False)
        d_col2=d_data2.filter(like='sample').columns[0]

        s_sub=s_name.split(' ')[1]
        d_sub=d_name.split(' ')[2]
        d_sub2=d_name2.split(' ')[2]


        print(f'Merging {s_name} samples with {d_name} samples')
        print('using columns')
        print(s_col,':',d_col)
        #data=pd.concat([s_data.set_index(s_col),d_data.set_index(d_col)], axis=1, join='inner')
        data=pd.merge(  
                s_data.drop_duplicates(subset=[s_col],keep='first'),
                d_data.drop_duplicates(subset=[d_col],keep='first'),
        left_on=s_col,
        right_on=d_col,
        how='outer',
        suffixes=['_'+s_sub,'_'+d_sub],
        )
        print(f'Merging {s_name} + {d_name} with {d_name} data')

        data=pd.merge(  
                        data.drop_duplicates(subset=[s_col],keep='first'),
                        d_data2.drop_duplicates(subset=[d_col2],keep='first'),
                        left_on=s_col,
                        right_on=d_col,
                        how='inner',
                        suffixes=['_'+s_name,'_'+d_name2[2]],
                        )

    data=ct.sort_data(data)
    data=ct.reorder_columns(data,verbose=True)
    data=ct.fix_depths(data)
    data=ct.reorder_columns(data)
    display(data.head(2))
    d_list.append(data)
    print('----------------------------')
    output=f'{dir}{basename}_merged.csv'
    print('output location:')
    print(output)
    try:
        data.to_csv(output,index=False)
    except Exception as e:
        print(e)


['geochemical', 'hyp-pkg', 'terraspec', 'xrf'] ['drill samples master.csv']
---------------------------------------------------
['drill geochemical assays master.csv', 'drill samples master.csv']
drill geochemical
Merging drill samples master  with drill geochemical assays master data
using columns
sample_id ; sample_id_description
first columns before sort
['sample_id', 'hole_id', 'from_ft', 'to_ft', 'from_m', 'to_m', 'description', 'qaqc', 'source_master', 'ag_ppm', 'ag_ppm(2)', 'al_%', 'as_ppm', 'au_ppm', 'au_ppm(2)']
first columns after sort
['sample_id', 'hole_id', 'from_ft', 'to_ft', 'from_m', 'to_m', 'description', 'qaqc', 'source_master', 'ag_ppm', 'ag_ppm(2)', 'al_%', 'as_ppm', 'au_ppm', 'au_ppm(2)']


,sample_id,hole_id,from_ft,to_ft,from_m,to_m,description,qaqc,source_master,ag_ppm,...,th_ppm,ti_%,tl_ppm,u_ppm,v_ppm,w_ppm,y_ppm,zn_%,zn_ppm,zr_ppm
0,B300015,KM-20-01,0.0,0.0,0.0,0.0,0,STD CDN-ME-1410,KM-20-01.xlsx,71,...,<50,0.05,<50,<50,10,<50,0.0,0,37100,0.0
1,B300016,KM-20-01,469.5,472.0,143.1,143.9,Chlorite-sericite schist with concordant quart...,0,KM-20-01.xlsx,<1,...,<50,0.60,<50,<50,350,<50,0.0,0,140,0.0


----------------------------
output location:
/Users/sebastiancoombs/Library/CloudStorage/GoogleDrive-sebastianbcoombs@gmail.com/Shared drives/AMC Projects/_AZ_Kay/_Master Databases/drill geochemical_merged.csv
---------------------------------------------------
['drill hyp-pkg samples master.csv', 'drill hyp-pkg certificates master.csv', 'drill samples master.csv']
drill hyp-pkg
Merging drill samples master samples with drill hyp-pkg samples master samples
using columns
sample_id : sample_id
Merging drill samples master + drill hyp-pkg samples master with drill hyp-pkg samples master data
Error in sorting
The column label 'hole_id_samples' is not unique.
first columns before sort
['sample_id', 'hole_id_samples', 'from_ft', 'to_ft', 'from_m', 'to_m', 'description_samples', 'qaqc', 'source_samples', 'hole_id_samples', 'depth_ft', 'depth_m', 'description_samples', 'source_samples', 'depthfrom']
first columns after sort
['sample_id', 'from_ft', 'to_ft', 'from_m', 'to_m', 'depth_ft', 'dept

,sample_id,from_ft,to_ft,from_m,to_m,depth_ft,depth_m,depthfrom,depthto,hole_id_samples,...,wd_mgohcb,wd_oh1400,whitemica_sc,xt_kaol2160,xt_kaol2180,xt_kaolinite,xt_wtmica,zeolite_sc,zoisite_sc,zunyite_sc
0,B300024,1027.0,1032.0,313.0,314.6,NaN,NaN,0.0,0.0,KM-20-01,...,45.0,22.59,45.0,0.0,0.0,0,0.000,0.0,0.0,0.0
1,B300025,1059.0,1062.0,322.8,323.7,NaN,NaN,0.0,0.0,KM-20-01,...,47.8,23.75,85.0,0.0,0.0,0,6.011,0.0,0.0,0.0


----------------------------
output location:
/Users/sebastiancoombs/Library/CloudStorage/GoogleDrive-sebastianbcoombs@gmail.com/Shared drives/AMC Projects/_AZ_Kay/_Master Databases/drill hyp-pkg_merged.csv
---------------------------------------------------
['drill terraspec spectral master.csv', 'drill terraspec samples master.csv']
drill terraspec
Merging drill terraspec samples master  with drill terraspec spectral master data
using columns
file_name ; sample
first columns before sort
['sample_id', 'hole_id', 'depth_ft', 'date_samples', 'file_name', 'internal_id', 'notes', 'source_samples', 'user', 'alfemg', 'aloh', 'bestelevation', 'bestlatitude', 'bestlongitude', 'by']
first columns after sort
['sample_id', 'hole_id', 'depth_ft', 'date_samples', 'file_name', 'internal_id', 'notes', 'source_samples', 'user', 'alfemg', 'aloh', 'bestelevation', 'bestlatitude', 'bestlongitude', 'by']


,sample_id,hole_id,depth_ft,date_samples,file_name,internal_id,notes,source_samples,user,alfemg,...,starrating1,starrating2,starrating3,starrating4,starrating5,starrating6,userelevation,userlatitude,userlongitude,version
0,T27-020,KM-21-27,0.0,44761.0,30184_AMC_KayMine_HaloStandard-KM-21-27-S_3018...,0,QAQC blank,KM-21-27.xlsx,AA,0,...,3.0,0.0,0.0,0.0,0.0,0.0,0.0,Not Set,Not Set,2.3
1,T27-040,KM-21-27,0.0,44761.0,30184_AMC_KayMine_HaloStandard-KM-21-27-S_3018...,0,QAQC blank,KM-21-27.xlsx,AA,0,...,3.0,0.0,0.0,0.0,0.0,0.0,0.0,Not Set,Not Set,2.3


----------------------------
output location:
/Users/sebastiancoombs/Library/CloudStorage/GoogleDrive-sebastianbcoombs@gmail.com/Shared drives/AMC Projects/_AZ_Kay/_Master Databases/drill terraspec_merged.csv
---------------------------------------------------
['drill xrf assays master.csv', 'drill xrf samples master.csv']
drill xrf
Merging drill xrf samples master  with drill xrf assays master data
using columns
sample_id ; sample_id
first columns before sort
['sample_id', 'hole_id', 'depth_ft_samples', 'date_samples', 'internal_id', 'notes', 'source_samples', 'user_samples', 'depth_ft_assays', '11-16-2022_xrf_km-22-95', 'ag_concentration', 'ag_error1s', 'ag_user_factor_offset', 'ag_user_factor_slope', 'al_concentration']
first columns after sort
['sample_id', 'hole_id', 'depth_ft_samples', 'date_samples', 'internal_id', 'notes', 'source_samples', 'user_samples', 'depth_ft_assays', '11-16-2022_xrf_km-22-95', 'ag_concentration', 'ag_error1s', 'ag_user_factor_offset', 'ag_user_factor_sl

,sample_id,hole_id,depth_ft_samples,date_samples,internal_id,notes,source_samples,user_samples,depth_ft_assays,11-16-2022_xrf_km-22-95,...,y_user_factor_offset,y_user_factor_slope,zn_concentration,zn_error1s,zn_user_factor_offset,zn_user_factor_slope,zr_concentration,zr_error1s,zr_user_factor_offset,zr_user_factor_slope
0,18-000,KM-21-18,0.0,44768.0,26-18,CDN-BL-10,KM-21-18.xlsx,AA,NaN,0.0,...,0.0,1.0,29.0,1.0,0.0,1.0,122.0,1.0,0.0,1.0
1,18-020,KM-21-18,0.0,44768.0,26-39,CDN-BL-10,KM-21-18.xlsx,AA,NaN,0.0,...,0.0,1.0,29.0,1.0,0.0,1.0,130.0,1.0,0.0,1.0


----------------------------
output location:
/Users/sebastiancoombs/Library/CloudStorage/GoogleDrive-sebastianbcoombs@gmail.com/Shared drives/AMC Projects/_AZ_Kay/_Master Databases/drill xrf_merged.csv


In [65]:
data=d_list[3]


def fix_depths(data,like=['_m','_ft'],verbose=True):
    depths=[d for d in data.columns if d.lower().startswith('depth')]
    print(depths)
    depth_map={d:'_'.join(d.split('_')[:-1]) for d in depths if len(d.split('_'))>2 }
    print (depth_map)
    
    data=data.rename(columns=depth_map)

    for l in like:
        depths=data.filter(like=l,)
        cols=[c for c in depths.columns if c.endswith(l)]
        cols=list(set(cols))

        if verbose:
            print('Filling', cols)
        data[cols]=data[cols].replace(0,np.nan)
        depths=data[cols].copy()

        depths=depths.apply(lambda x: x.fillna(x.mean(skipna=True)),axis=1)
        data[cols]=depths
        # data.drop(data.columns.isduplicated(),axis=1)
    return data




['depth_ft_samples', 'depth_ft_assays']
{'depth_ft_samples': 'depth_ft', 'depth_ft_assays': 'depth_ft'}
Filling []
Filling ['depth_ft']


sample_id                False
hole_id                  False
depth_ft                 False
depth_ft                 False
date_samples             False
                         ...  
zn_user_factor_slope      True
zr_concentration         False
zr_error1s               False
zr_user_factor_offset     True
zr_user_factor_slope      True
Length: 187, dtype: bool

In [42]:
data[cols].apply(lambda x: x.fillna(x.mean(skipna=True)),axis=1)


NameError: name 'cols' is not defined

In [ ]:
data[cols]=data[cols].apply(lambda x: x.fillna(x.mean(skipna=True)),axis=1)
data[cols]


In [172]:

data=d_list[1].filter(like='_m')
cols=data.columns[:3]
data[cols]=data[cols].replace(0,np.nan)
data[cols]=data[cols].apply(lambda x: x.fillna(x.mean(skipna=True)),axis=1)
data[cols]


/var/folders/jc/lq6pqgbn3dlfs9v5g5x4jc_80000gn/T/ipykernel_57952/2890912278.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[cols]=data[cols].replace(0,np.nan)
/var/folders/jc/lq6pqgbn3dlfs9v5g5x4jc_80000gn/T/ipykernel_57952/2890912278.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[cols]=data[cols].apply(lambda x: x.fillna(x.mean(skipna=True)),axis=1)


,from_m,to_m,depth_m
0,143.1,143.9,143.50
1,313.0,314.6,313.80
2,278.4,279.8,279.10
3,281.5,282.4,281.95
4,269.4,270.4,269.90
...,...,...,...
9503,658.4,658.4,658.40
9504,661.4,661.4,661.40
9505,664.5,664.5,664.50
9506,667.5,667.5,667.50


In [ ]:

depths.columns=['_'.join(c.split('_')[:2]) for c in depths.columns]
d_cols=depths.columns
depths.columns.duplicated()
depths.T.drop_duplicates().T


In [ ]:
depths=depths.T.drop_duplicates(keep='first').T
df.groupby("name").transform(lambda x: x.fillna(x.sum))


In [ ]:
depths=depths.replace(0,np.nan)
depths.columns=['_'.join(c.split('_')[:2]) for c in depths.columns]
depths=depths.T.drop_duplicates(keep='first').T
df.groupby("name").transform(lambda x: x.fillna(


In [91]:
def clean_for_merge(data):
    for col in data.columns:
        if 'depth_ft'in col.lower():
            try:
                data[col]=pd.to_numeric(data[col],errors='coerce')
            except Exception as e:
                print(e)
            data['from_ft']=data[col]
            data['to_ft']=data['from_ft'].shift(-1)-.5
            #convert ft to meters
            data['from_m']=data['from_ft']* .3281
            data['to_m']=data['to_ft'] * .3281
            data=data.drop(data.filter(like='Depth').columns,axis=1)
        if ('hole' in col.lower())& ('id' in col.lower()):
            data[col]=data[col].astype(object)
            data.rename(columns={f'{col}':f'hole_id'},inplace=True)
        try: 
            data[col]=data[col].str.strip()
        except Exception as e:
            pass
            #print(f'{col}: {e}')
        data.rename(columns={f'{col}':f'{col.strip().lower()}'},inplace=True)
        data=data.drop(data.filter(like='unnamed'),axis=1)
        if 'from_ft'in col.lower():
            try:
                drop_index=data[data['from_ft'].isna()].index
                data=data.drop(drop_index,axis=0)
            except Exception as e:
                pass
                #print(f'{e}')
    if 'hole_id' not in data.columns:
        data['hole_id']=np.nan
    if 'recvd wt.'  in data.columns:
        try:
            data['recvd wt.']=pd.to_numeric(data['recvd wt.'],errors='coerce')
            data['from_ft']=pd.to_numeric(data['from_ft'],errors='coerce')
        except Exception as e:
            print(e)
            pass
    names=data.loc[:,data.columns.duplicated()].columns
    if len(names)>0:
        print(f'duplcated column: {names} drop or else the nasty merge bug')
        data=data.loc[:,~data.columns.duplicated()].copy()

    return data

In [92]:
def fill_merge_groups(data):
    data=data.fillna(method='ffill')
    data=data.fillna(method='bfill')
    data=data.drop_duplicates(keep='first')
    return data

In [93]:
def explode_depths(data):
    data[['from_ft','to_ft']]=data[['from_ft','to_ft']].fillna(0)
    if 'sample_id' in data.columns:
        zipped=zip(data['sample_id'],data['hole_id'], data['from_ft'], data['to_ft'])
        depths=pd.DataFrame([(s_id,h_id, y) for s_id,h_id, start, end in zipped for y in np.arange(start, end,.5)],
                    columns=['sample_id','hole_id','true_depth'])
        d_frame=depths.merge(data,on=['sample_id','hole_id'])
        d_frame=depths.merge(data,on=['sample_id','hole_id'])
    else:
        data['START']=data['from_ft']
        zipped=zip(data['START'],data['hole_id'], data['from_ft'], data['to_ft'])
        depths=pd.DataFrame([(s,h_id, y) for s,h_id, start, end in zipped for y in np.arange(start, end,.5)],
                        columns=['START','hole_id','true_depth'])
        d_frame=depths.merge(data,on=['START','hole_id'])
        d_frame=d_frame.drop('START',axis=1)
    d_frame=d_frame.drop(d_frame.filter(like='from').columns,axis=1)
    d_frame=d_frame.drop(d_frame.filter(like='to').columns,axis=1)
    return d_frame

In [ ]:
def pull_start_end(data):
    depths=data['true_depth']
    start=depths.values[0]
    end=depths.values[-1]
    data['from_ft']=start
    data['to_ft']=end
    data=data.drop_duplicates(subset=['from_ft','to_ft'],keep='first')
    return data

In [ ]:
hole_list=[]
sample_list=[]
sample_data_names=[]
hole_data_names=[]
files=[file for file in os.listdir(fconfig.output_path) if file.endswith('.csv')]
for file in files:
    name=file.split('.')[0]
    if len(name)==0:
        continue
    print(f'read {name}')
    try:
        data=pd.read_csv(fconfig.output_path+file,low_memory=False,on_bad_lines='skip')

    except Exception as e:
        print(e)
        print('likely not a csv')
        continue
    print(f'clean {name}')
    data=clean_for_merge(data)

    if 'sample_id' in data.columns:
        print(f'{name} has sample ids')
        sample_list.append(data)
        sample_data_names.append(name)
    else:
        print(f'{name} does not have sample ids')
        print(f'explode {name}')
        try:
            data=explode_depths(data)
            hole_list.append(data)
            hole_data_names.append(name)
        except Exception as e:
            print(e)
            print('must have (depths and hole ids) or (sample_ids) to merge into master')

In [ ]:
# merge the data in each group. holes and samples
def merge_dflist(data_list,data_names):
    big_df=pd.DataFrame()
    non_id_cols=['true_depth', 'hole_id']
    for i,data in enumerate(data_list):
        ## merge all the data with sample_ids
        name=data_names[i]
        suf=name.split(' ')[0]
        print(f"####### merge frame {i}: {name} suffix= {suf} #######")
        data=data.astype(object)
        big_df=big_df.astype(object)
        big_cols=list(big_df.columns)
        small_cols=list(data.columns)
        
        dups=sorted([col for col in big_cols if col in small_cols])
        cols=config.main_columns
        try:
            [dups.remove(x) for x in ['file','folder','start_depth','end_depth']if x in data.columns]
        except:
            pass
        if 'sample_id' not in data.columns:
            cols=non_id_cols
            try:
                big_index=list(big_df.hole_id.unique())
                small_index=list(data.hole_id.unique())
                same_ids= [id for id in big_index if id in small_index]
            except Exception as e:
                print(e)
                same_ids=[]
            dups=non_id_cols
        else:
            big_index=list(big_df.index)
            small_index=list(data.index)
            same_ids= [id for id in big_index if id in small_index]
        print(f'common columns {dups}')
        n_ids=len(same_ids)
        if n_ids==0:
            print(f'{n_ids} common ids concat')
            big_df=pd.concat([big_df,data],axis=0,join='outer')
        else:
            print(f'{n_ids} common ids merge')
            big_df=big_df.merge(data,on=dups,suffixes=[None,'_'+suf],how='outer')

        print(f'data shape {big_df.shape}')
    
    not_dups=[i for i in big_df.columns if i not in cols]
    big_final=pd.concat([big_df[cols],big_df[not_dups]],axis=1)

    big_final.drop(big_final.filter(like='file').columns,axis=1,inplace=True)
    big_final.drop(big_final.filter(like='folder').columns,axis=1,inplace=True)
    big_final.sort_index(inplace=True)
    return big_final

In [ ]:
big_samples=merge_dflist(sample_list,sample_data_names)
print('FINISHED MERGING SAMPLES')
big_holes=merge_dflist(hole_list,hole_data_names)

In [ ]:
data=big_samples
def list_fill(data,column,id):
    data=data[data[column]==id].fillna(method='ffill').fillna(method='bfill').drop_duplicates(keep='first')
    return data.copy()
def fast_fill_merge(data,column,desc):
    dup_id=list(data[data[column].notna().duplicated()][column].unique())
    data_list=[list_fill(data,column,id) for id in tqdm(dup_id,desc='Merging on depth')]
    print('done listing')
    data=pd.concat(data_list,axis=0)
    print('done with concat')
    return data

In [ ]:
def list_start_end(data,column,id):
    data=data[data[column]==id]
    depths=data['true_depth']
    data=data.drop('true_depth',axis=1)
    data=data.drop_duplicates(keep='first')
    start=depths.values[0]
    end=depths.values[-1]
    data['from_ft']=start
    data['to_ft']=end
    data=data.drop_duplicates(keep='first').copy()
    return data
    
def fast_pull_ends(data,column):
    dup_id=list(data[data[column].notna().duplicated()][column].unique())
    data_list=[list_start_end(data,column,id) for id in tqdm(dup_id,desc='pulling from/too depths:')]
    print('done listing data with ends')
    data=pd.concat(data_list,axis=0)
    print('done with concat')
    return data

In [ ]:
print('#####clean the sample_ids#########')
big_samples=fast_fill_merge(data,column='sample_id',desc='clean_samples')
big_samples



In [ ]:
print('#####explode sample_ids#########')
samp_ex=explode_depths(big_samples)
print('######### merge holes and samples ##########')
merged=samp_ex.merge(big_holes,on=['hole_id','true_depth'],how='outer')
print('######### fill the final_data ##########')
merged['u_id']=merged.fillna(np.inf).groupby(['hole_id','true_depth']).ngroup()

In [ ]:
merged_test=fast_fill_merge(merged,column='u_id',desc='clean total_data')


In [ ]:
merged_test['u_id']=merged_test.fillna(np.inf).drop('true_depth',axis=1).groupby(merged_test.drop('true_depth',axis=1).columns.to_list()).ngroup()
len(merged_test['u_id'].unique())

In [ ]:
merged_test=fast_pull_ends(merged_test,column='u_id')
final_cols=['sample_id','hole_id','from_ft','to_ft']
des_cols = merged_test.filter(like='description').columns
other_cols= [col for col in merged_test.columns if col not in final_cols]
merged_test=pd.concat([merged_test[final_cols],merged_test[des_cols],merged_test[other_cols]],axis=1)
merged_test

In [ ]:
merged_test

In [ ]:
end

In [ ]:
merged['u_id']=merged.fillna(np.inf).groupby(merged.drop('true_depth',axis=1).columns.to_list()).ngroup()

In [ ]:

merged_test=fast_pull_ends(merged,column='u_id',desc='pull start ends')
final_cols=['sample_id','hole_id','from_ft','to_ft']
des_cols = merged_test.filter(like='description').columns
other_cols= [col for col in merged_test.columns if col not in final_cols]
merged_test=pd.concat([merged_test[final_cols],merged_test[des_cols],merged_test[other_cols]],axis=1)
merged_test



In [ ]:
merged['u_id']=merged.fillna(np.inf).groupby(merged.drop('true_depth',axis=1).columns.to_list()).ngroup()

In [ ]:
merged['from_ft']=np.inf
merged['to_ft']=np.inf
merged

In [ ]:
merged_test=merged.groupby(merged['u_id']).progress_apply(pull_start_end).reset_index(drop=True)

In [ ]:
merged_test.drop('true_depth',axis=1, inplace=True)

In [ ]:
final_cols=['sample_id','hole_id','from_ft','to_ft']
des_cols = merged_test.filter(like='description').columns
other_cols= [col for col in merged_test.columns if col not in final_cols]
merged_test=pd.concat([merged_test[final_cols],merged_test[des_cols],merged_test[other_cols]],axis=1)
merged_test

In [ ]:
merged_test=merged_test.T.drop_duplicates(keep='first').T
merged_test

In [ ]:

for col in merged_test.columns:
    if merged_test[col].isna().sum()==len(merged_test):
        print(f'drop {col}: no data')
        merged_test=merged_test.drop(col,axis=1)
merged_test=merged_test.sort_values(by=['sample_id','hole_id','from_ft'],ascending=False)
merged_test=merged_test.set_index('sample_id')
merged_test

In [ ]:
merged_test.to_excel('/Volumes/GoogleDrive/Shared drives/AMC Projects/_AZ_Kay/_Master Databases/master_MASTER.xlsx')